<a href="https://colab.research.google.com/github/choimiree/TIL/blob/master/esports_data_analysis_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 이스포츠 데이터 분석 with 배틀그라운드 이스포츠 데이터



가장 먼저 해볼 내용은 Python 환경에서 MySQL - Pandas를 연동하여 데이터를 받아오는 부분입니다.  
실제로도 현업에서 많이 쓰는 부분이기도 합니다. 쿼리를 통해서 최종 분석 결과물을 내는데에는 한계가 있기떄문에 SQL 서버와 분석 환경을 연동하여 분석을 진행합니다. 

먼저 Python 환경에서 MySQL을 사용할 수 있도록 pymysql 라이브러리를 설치합니다

In [1]:
!pip install pymysql

     |████████████████████████████████| 51kB 1.5MB/s 


그 다음 pandas와 pymysql을 import 합니다.  
오늘 사용할 SQL 서버의 정보는 아래와 같습니다. 

Host : `pubg-esports-match-stats.ciaedta3cwyj.ap-northeast-2.rds.amazonaws.com`  
User : `dataitgirls`  
Password : `이따가 알려줄게여`  

`pymysql.connect` 함수에 위의 정보를 입력하고 연결을 실행합니다. 

In [2]:
import pandas as pd
import numpy as np
import pymysql

In [3]:
conn = pymysql.connect(host = 'pubg-esports-match-stats.ciaedta3cwyj.ap-northeast-2.rds.amazonaws.com',
                       port = 3306,
                       user = 'dataitgirls',
                       password = 'epdltrjfwm',
                       db = 'pubg-esports')

기존에 배웠던 SQL 문법을 사용하여 자유롭게 아래 

*   항목 추가
*   항목 추가

데이터를 가지고 놀아봅시다!

In [4]:
sql_input = "select * from esports_matches_summary_stats"
df_esports_matches_summary_stats = pd.read_sql_query(sql_input, conn)

데이터 셋 안에는 경기 결과에 대한 정보가 들어 있습니다.  
`player_id` : 인게임 선수 아이디   
`player_name` : 선수 이름  
`rank` : 순위  
`kills` : 킬 수   
`assists` : 어시스트 수   
`damage` : 상대에게 가한 피해량  
`time_servived` : 생존 시간  
`headshot_kills` : 헤드샷 횟수  
`ride_distance` : 챠량으로 이동한 거리    
`walk_distance` : 걸어서 이동한 거리   
`revives` :  상대를 부활시킨 횟수  
`DBNOs` :  상대를 기절시킨 횟수  
`heals` :  치료제 사용 개수  
`boosts` : 부스트제 사용 개수     

In [6]:
df_esports_matches_summary_stats.head()

,player_id,player_name,rank,kills,assists,damage,time_survived,headshot_kills,ride_distance,walk_distance,revives,DBNOs,heals,boosts
0,AFK_APXB,APXB,5,1,1,148.77348,988.608,0,3369.1400,5964.8706,0,2,9,4
1,AFK_Asag2,ASAG2,5,0,1,23.34000,1004.771,0,6476.3770,2274.9470,0,1,7,5
2,AFK_Caley,CALEY,5,2,0,163.14350,1229.768,1,6084.7236,2510.9750,0,0,10,6
3,AFK_Jaelong,JAELONG,5,0,0,0.00000,814.769,0,6634.0684,1613.8353,0,0,7,5
4,EZ_Donghyun,DONGHYUN,8,2,3,162.80000,925.875,1,1309.7083,3221.3508,0,1,1,1


In [7]:
pd.read_sql_query('select player_name from esports_matches_summary_stats', conn)

,player_name
0,APXB
1,ASAG2
2,CALEY
3,JAELONG
4,DONGHYUN
...,...
8609,SPEAR
8610,DG98
8611,FOXY
8612,HULK


3개 대회 동안 많게는 100경기 정도를 치른 선수도 있습니다 

In [ ]:
df_player_matches = df_esports_matches_summary_stats.groupby('player_name').size().to_frame('matches').sort_values(by = 'matches', ascending = False)
df_player_matches.head(10)

,matches
player_name,
AYOH,100
SILKY,100
DUMBO,100
DG98,90
HIKARI,90
LECLO,90
KAYLE,90
SEOUL,90
SEONGJANG,90


In [10]:
df_player_matches = df_esports_matches_summary_stats.groupby('player_name').size().to_frame('matches').sort_values(by='matches', ascending=False)
df_player_matches.head(10)

,matches
player_name,
AYOH,100
SILKY,100
DUMBO,100
DG98,90
HIKARI,90
LECLO,90
KAYLE,90
SEOUL,90
SEONGJANG,90


가장 전통적(?)으로 사용하는 지표는 `kills`과 `damage` 를 합산 하는 지표입니다. 간단하게 선수의 능력을 나타낼 수 있습니다.

In [12]:
df_player_summary_stats = df_esports_matches_summary_stats.groupby('player_name')["kills", "damage"].sum().sort_values(by = "damage", ascending = False)
df_player_summary_stats.head(10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,kills,damage
player_name,,
PIO,167,27513.256234
UNDER,128,26872.345104
2HEART,118,25119.963188
HIKARI,117,22962.286727
AYOH,113,22175.286903
INDIGO,134,21527.709863
RENBA,110,21507.650665
KAYLE,91,21292.086822
SEOUL,122,19808.067148


In [13]:
df_player_summary_stats_adj = pd.merge(df_player_matches, df_player_summary_stats, how= 'left', on= 'player_name')
df_player_summary_stats_adj.assign(kills_per_matches = df_player_summary_stats_adj['kills'] / df_player_summary_stats_adj['matches']).sort_values(by = 'kills_per_matches', ascending = False)

,matches,kills,damage,kills_per_matches
player_name,,,,
PIO,90,167,27513.256234,1.855556
KEDIYA,5,9,1171.956428,1.800000
BBABBO,10,17,3272.291161,1.700000
CRUX,20,32,4220.444319,1.600000
ORANGE,5,8,537.832324,1.600000
...,...,...,...,...
CHEONGHO,5,0,107.807245,0.000000
OHJEON,10,0,523.281582,0.000000
FREE,4,0,98.808989,0.000000


In [36]:
df_player_summary_stats_adj.query('matches >= 50').assign(kills_per_matches = df_player_summary_stats_adj['kills'] / df_player_summary_stats_adj['matches'],
                                                          damage_per_matches = df_player_summary_stats_adj['damage']/ df_player_summary_stats_adj['matches']).sort_values(by = 'kills_per_matches',
                                                                                                                                                                          ascending = False)

,matches,kills,damage,kills_per_matches,damage_per_matches
player_name,,,,,
PIO,90,167,27513.256234,1.855556,305.702847
INDIGO,90,134,21527.709863,1.488889,239.196776
UNDER,90,128,26872.345104,1.422222,298.581612
SEOUL,90,122,19808.067148,1.355556,220.089635
MINSUNG,90,121,18958.017746,1.344444,210.644642
...,...,...,...,...,...
KIMGONI,80,51,14225.255976,0.637500,177.815700
CHEONGGAK,60,32,7434.871675,0.533333,123.914528
PARKCHAN,55,29,7930.373127,0.527273,144.188602



하지만 각 팀마다의 출전 횟수가 차이가 있습니다. 그걸 해결하기 위해 `경기 당 kills` , `경기 당 damage` 지표가 만들어졌습니다.

In [39]:
df_player_summary_stats_adj = pd.merge(df_player_matches, df_player_summary_stats, how= 'left', on = 'player_name')
df_player_summary_stats_adj.assign(kills_per_matches = df_player_summary_stats_adj['kills'] / df_player_summary_stats_adj['matches'],
                                   damage_per_matches = df_player_summary_stats_adj['damage'] / df_player_summary_stats_adj['matches']).sort_values(by = "kills_per_matches",
                                                                                                                                                    ascending = False)

,matches,kills,damage,kills_per_matches,damage_per_matches
player_name,,,,,
PIO,90,167,27513.256234,1.855556,305.702847
KEDIYA,5,9,1171.956428,1.800000,234.391286
BBABBO,10,17,3272.291161,1.700000,327.229116
CRUX,20,32,4220.444319,1.600000,211.022216
ORANGE,5,8,537.832324,1.600000,107.566465
...,...,...,...,...,...
CHEONGHO,5,0,107.807245,0.000000,21.561449
OHJEON,10,0,523.281582,0.000000,52.328158
FREE,4,0,98.808989,0.000000,24.702247


In [ ]:
조금 더 자세한 능력을 얻기 위해서 추가 통계 정보를 가져옵니다

In [41]:
sql_input = "select * from esports_matches_shooting_stats"
df_esports_matches_shooting_stats = pd.read_sql_query(sql_input, conn)

데이터 셋 안에는 경기 결과에 대한 정보가 들어 있습니다.
`player_id` : 인게임 선수 아이디  
`player_name` : 선수 이름  
`attack` : 1경기 당 공격 횟수  
`take_damage` : 1경기 당 상대 명중 횟수  
`total_damage_by_gun` : 총기를 사용하여 상대를 명중시킨 횟수   
`HeadShot` : 명중 부위 - 머리   
`ArmShot` : 명중 부위 - 팔  
`LegShot` : 명중 부위 - 다리  
`PelvisShot` : 명중 부위 - 골반  
`TorsoShot` : 명중 부위 - 몸통 

In [42]:
df_esports_matches_shooting_stats.head()

,player_id,player_name,attack,take_damage,total_damage_by_gun,HeadShot,ArmShot,LegShot,PelvisShot,TorsoShot
0,AFK_APXB,APXB,25,6.0,148.773479,0.0,0.0,0.0,1.0,5.0
1,AFK_Asag2,ASAG2,29,1.0,23.340000,0.0,0.0,0.0,1.0,0.0
2,AFK_Caley,CALEY,43,8.0,163.143497,1.0,0.0,1.0,1.0,5.0
3,EZ_Donghyun,DONGHYUN,70,16.0,162.800005,2.0,0.0,1.0,5.0,8.0
4,EZ_GGangsu,GGANGSU,83,19.0,390.843147,6.0,1.0,0.0,1.0,11.0


조금 더 자세한 지표를 만들어봅니다!

`damage_rate` = `total_damage_by_gun` / `attack`    
`active_damage_rate` = `total_damage_by_gun` / `take_damage`  
`shooting_accuracy` = `take_damage` / `attack`  

In [56]:
df_esports_matches_shooting_stats_adj = df_esports_matches_shooting_stats.groupby('player_name')['attack', 'take_damage', 'total_damage_by_gun'].sum()
        
df_result = df_esports_matches_shooting_stats_adj.assign(damage_rate = df_esports_matches_shooting_stats_adj['total_damage_by_gun'] / df_esports_matches_shooting_stats_adj['attack'],
                                                   active_damage_rate = df_esports_matches_shooting_stats_adj['total_damage_by_gun'] / df_esports_matches_shooting_stats_adj['take_damage'],
                                                   shooting_accuracy = df_esports_matches_shooting_stats_adj['take_damage'] / df_esports_matches_shooting_stats_adj['attack'])
df_result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,attack,take_damage,total_damage_by_gun,damage_rate,active_damage_rate,shooting_accuracy
player_name,,,,,,
1NTIV,3366,234.0,4514.927977,1.341333,19.294564,0.069519
2HEART,8194,1039.0,23263.937827,2.839143,22.390701,0.126800
2MAE,948,101.0,2229.645130,2.351946,22.075694,0.106540
2TAP,4076,398.0,10081.726439,2.473436,25.330971,0.097645
ADDER,6346,763.0,15016.690283,2.366324,19.681114,0.120233
...,...,...,...,...,...,...
YUNJONG,8970,841.0,17674.968699,1.970454,21.016610,0.093757
ZAMIL,1731,124.0,2619.088759,1.513050,21.121684,0.071635
ZED,1142,67.0,1521.314034,1.332149,22.706180,0.058669


In [34]:
df_total_stats = pd.merge(df_player_summary_stats_adj, df_result, how = 'left', on='player_name')
df_total_stats.query('matches >= 50').sort_values(by = 'active_damage_rate', ascending = False)

,matches,kills,damage,attack,take_damage,total_damage_by_gun,damage_rate,active_damage_rate,shooting_accuracy
player_name,,,,,,,,,
2TAP,70,35,10262.463764,4076,398.0,10081.726439,2.473436,25.330971,0.097645
KIMGONI,80,51,14225.255976,5565,575.0,14185.781020,2.549107,24.670924,0.103324
PIKY,60,52,11189.310853,4256,438.0,10735.423455,2.522421,24.510099,0.102914
YACHA,90,80,18058.228448,7534,727.0,17028.437762,2.260212,23.422886,0.096496
LECLO,90,65,14765.838992,6678,614.0,14215.344196,2.128683,23.152026,0.091944
UNTOP,50,34,8226.501990,3404,336.0,7746.040141,2.275570,23.053691,0.098707
2HEART,90,118,25119.963188,8194,1039.0,23263.937827,2.839143,22.390701,0.126800
MIHA,50,52,10697.050451,5870,439.0,9803.055642,1.670027,22.330423,0.074787
MACZORA,90,90,19132.463113,7781,804.0,17935.600245,2.305051,22.307961,0.103329
